In [ ]:
import pickle
import numpy as np
from collections import Counter
from model import Board, RandomPlayer, MiniMaxPlayer, HumanPlayer, Player, search, minimax, draw_graph
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

# Set up a player and blank board

In [ ]:
# X goes first
player = Player('X')
board = Board(state=np.zeros((3,3), dtype=int))
board.draw(board.state)

# Minimax over full game space (depth=9)

In [ ]:
%%time
G = search(board, player_marker='X', max_depth=9, player_moves_first=True)

In [ ]:
print(G.number_of_nodes())
leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
print(len(leaf_nodes))

In [ ]:
#Gs = G.subgraph(nx.dfs_tree(G, 0))
Gs = G.subgraph(nx.dfs_tree(G, 0, depth_limit=3))
Gs.number_of_nodes()
draw_graph(Gs, fig_size=(5, 20), node_label='player', edge_label='move')

# Do minimax over all states

In [ ]:
%%time
H = minimax(G, first_move=True)

In [ ]:
#Gs = G.subgraph(nx.dfs_tree(G, 0))
Hs = H.subgraph(nx.dfs_tree(H, 0, depth_limit=2))
Hs.number_of_nodes()
draw_graph(Hs, fig_size=(5, 20), node_label='player', edge_label='move')

In [ ]:
#Gs = G.subgraph(nx.dfs_tree(G, 0))
Hs = H.subgraph(nx.dfs_tree(H, 800, depth_limit=4))
Hs.number_of_nodes()
draw_graph(Hs, fig_size=(5, 20), node_label='score', edge_label='move')

In [ ]:
# Add best minimax move to each node
for n in H.nodes():
    scores = [(v, c['move'], H.nodes[v]['score']) for (u, v, c) in H.out_edges(n, data=True)]
    if scores:
        best_move = max(scores, key = lambda t: t[2])[1]
        H.nodes[n]['best_move'] = best_move

In [ ]:
# Hash all best moves:
minimax_move = {}

for node in H.nodes(data=True):
    if node[1].get('best_move') and node[1].get('player') == 'max':
        minimax_move[node[1]['state'].tostring()] = node[1]['best_move']

## Save minimax dictionary to disk

In [ ]:
with open('ttt_minimax_first_move_dict.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(minimax_move, f, pickle.HIGHEST_PROTOCOL)